## Introduction

Explore and cluster the neighborhoods in Toronto.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Environment Setup</a>

2. <a href="#item1">Download and Explore Dataset</a>

3. <a href="#item2">Explore Neighborhoods in Toronto</a>

4. <a href="#item3">Analyze Each Neighborhood</a>

5. <a href="#item4">Cluster Neighborhoods</a>

6. <a href="#item5">Examine Clusters</a>    
</font>
</div>

### 1. Environment Setup

Environment Setup.

In [1]:
#Install lxml to parse the html
!conda install -c anaconda beautifulsoup4

!conda install -c anaconda lxml
print('Beautifulsoup & lxml Installed.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 anaconda

beautifulsoup4 100% |################################| Time: 0:00:00  42.87 MB/s


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  27.03 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  38.16 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  54.48 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  37.64 MB/s
vincent-0.4.4- 100% |###################

### 2. Download and Explore Dataset

Obtain the Wiki HTML page using Beautifulsoup.

In [72]:
from bs4 import BeautifulSoup

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

Extract the postal information for Canada

In [77]:
#Assign the column headers
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

data = []

#Find the postal table 
table = soup.find('table', class_='wikitable sortable')
table_body = table.find('tbody')
rows = table_body.find_all('tr')

#Parse each rows to extract the data
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    
    if len(cols) > 0 and cols[1] != 'Not assigned':      # Only interested Borough not equal to 'Not assigned' 
        if cols[2] == 'Not assigned':
            cols[2] = cols[1]                            # Overlay Borough if Neighborhood is 'Not sssigned'
        data.append([ele for ele in cols if ele])


neighborhoods = pd.DataFrame(data, columns=column_names)

# For duplicate postal code merge Neighborhoods
toronto_neighborhoods = neighborhoods.groupby('PostalCode')\
  .agg({'Borough' : 'first', 'Neighborhood' : ','.join})\
  .reset_index().reindex(columns=neighborhoods.columns)

#print the toronto neighborhood

toronto_neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [80]:
#print the shape of the dataframe
toronto_neighborhoods.shape

(103, 3)